# for문으로 크롤링 : 1페이지만
## 환경 설정

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
#import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 지역 키워드를 입력하기
keyword = '횟집'

# page 정하기
page = 1

In [3]:
# 크롬창 띄우기
driver = webdriver.Chrome('./chromedriver')

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [4]:
# 광고창 iframe 접근하기
driver.switch_to.frame("google_ads_iframe_/395211568/init/desktop_all_0")

# '다시 보지 않기' 버튼 클릭하기
element = '#ad > div > button.ad_btn.ad_block_btn'
driver.find_element(By.CSS_SELECTOR, element).click()

In [5]:
driver.switch_to.default_content()

## 크롤링 함수

In [19]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    up_height = 2700
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.find_element(By.CSS_SELECTOR,element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR,element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            try:
                # 음식 종류 및 메뉴 크롤링
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
                food_type = driver.find_element(By.CSS_SELECTOR,element1)
                food_type = food_type.text
                #food_type = type_raw
            except:
                pass
            
            try:
                element2 = 'span.Restaurant_Menu'
                menu_elements = driver.find_elements(By.CSS_SELECTOR,element2)
                food_menu = [menu_element.text for menu_element in menu_elements]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            sub_dict['Menu'] = food_menu
            
            
            new_height = driver.execute_script('return document.body.scrollHeight') #처음 스크롤 길이 반환
            while True:
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #쭉 내려갔다.
                time.sleep(0.5)
                resize = new_height- up_height # 길이 계산
                driver.execute_script(f"window.scrollTo(0, {resize})") #웹드라이버 반환
                
                try:
                    more_btn = driver.find_element(By.CSS_SELECTOR,'div.RestaurantReviewList__MoreReviewButton')
                    driver.implicitly_wait(3)
                    time.sleep(0.5)
                    more_btn.click()
                    time.sleep(1)
                    last_height = driver.execute_script('return document.body.scrollHeight')
                    if last_height == new_height:
                        break
                    else:
                        new_height = last_height
                        
                except: #더보기 버튼이 없는 경우
                    break
            
            review_list = list()
            
            try:
                review_elements = driver.find_elements(By.CSS_SELECTOR,'p.RestaurantReviewItem__ReviewText')
                sleep(1)
                for ele in review_elements:
                    review_list.append(ele.text)
            except:
                pass
            
            sub_dict['review'] = review_list
        
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    # 데이터 프레임 생성, 키값을 열레이블로\
    return df, total_dict

In [18]:
df, total_dict = FoodCrawling(row = 1, col = 1)
df

월드베스트 고등어회다. 먹선생 기준 미슐랭 3스타. 제주도 가면 반드시 가야하는 식당, 아니? 여기 고등어회 먹으러 제주도 또갈거다.
-
제주 서귀포시 모슬포 #미영이네 
-
미영이누나(?)에게 사랑을 고백하고싶어지는 곳이에요. 스토리에 올리자마자 디엠이 폭발했었죠. 부럽다부터 시작해서 여기가 내 인생고등어회라는 말까지 다들 미영이네에 대한 사랑이 묻어나는 메세지를 보내주시더라구요.
-
#고등어회
한마디로 그냥 “인생고등어회”입니다. 맛있어요. 뭔지 모르겠는데 그냥 맛있어요. 아무것도 안찍어먹어도 맛있고 각종 소스에 곁들여먹어도 맛있어요. 맛있다는 말밖에할 수 없어 죄송하지만 그냥 맛있는걸 어떡해요?
-
인기가 많은 이유가 있어요. 단지 맛있다는 이유만으로도 충분한데 모든 직원분들이 각자의 개성대로 친절하고 유쾌하셔서 식사하는 시간 내내 기분이 좋았습니다.
-
당시 저녁시간보다 살짝 일찍 맞춰 갔음에도 불구하고 마감이였는데 감사하게도 먹선생은 막차타고 문닫고들어가는 행운아.
-
고등어회가 정말 맛있고 술이랑 같이 먹으면 한라산 정상이 제 뱃속이 들어있다고 할 정도로 한도끝도없이 들어갈것 같습니다. 운전때문에 참았어요. 여러분 미영이네 갈때는 꼭 택시타고 가세요.
-
아참. 고등어탕이 진짜 맛있어요. 맑은 탕으로 나오는데 하나도 비리지 않고 정말 깊은 맛이 납니다. 다음 제주도 방문때는 미영이네에 오기위해 숙소를 미영이네 근처로 잡으려구요. 최소한 서귀포에라도.
-
총평
WORLD BEST
고등어회
원래 가던 곳보다 두껍게 썰어준다
고등어탕은 처음 먹어보는데 고소하고 맛있다
앞에 바로 항구가 있어서 진짜 섬에 온 느낌이 나서 좋았다
제주도 갈때마다 방문해서... 몇번째 방문인지도 모르겠는 미영이네. 근데 정말 갈때마다 맛있다ㅠ.ㅠ

친구랑 내가 고등어 회를 너무 좋아해서 제주 여행다니다가도 좀 괜찮다는 곳 있으면 들어가서 먹어보고 했는데 도대체 미영이네보다 맛있는 곳이 없었음.. 이유가 뭔데 여기 고등어는 이렇게 기름지고 고소하고 맛잇는지가 너무 궁금하다...!

이 고등어는 무슨일이죠?…오마카세에 나오는 심해사바도 미영이네 고등어 앞에서는 사바사바 하며 고소함의 비결을 물어봐야합니다.

아무리 제주도에 맛있고 싱싱하게 한다는 고등어 회집을 가도 고등어는 그래 고등어지.. 등푸른 생선 특유의 비린 맛이 없을 수가 없잖아 하면서 가만하고 먹었는데, 비린 맛은 전혀 찾을 수 없고 통깨 부셔 놓은 것 마냥 고소한 풍미만 폭발합니다.

제발 메뉴판에 사장님이 정성껏 적어 주신 고등어회 맛있게 드시는 법을 따라서 드셔주세요. 매콤 상콤한 미나리와 양파가 쌈장과 함께 김과 회를 적셔주며 굉장한 시너지를 일으킵니다.

고등어탕은 제가 사진을 너무 못 찍었지만 드셔보세요.. 먹으면서 속풀리고 해장 되는 맛. 비교 할 수 없는 시원함이 대박입니다. JMT 제주도 맛집 인정.
고등어 회+탕 (대 3-4인분, 77.0)

제주도에서 반드시 먹어야겠다고 생각했던 미영이네식당의 고등어회와 고등어 탕. 고등어회는 비려서 잘 못먹는 경우가 많은데, 미영이네에서 그런 걱정은 하지 않아도 된다. 회가 정말 비리지 않아서 고등어 본연의 고소한 맛을 제대로 느낄 수 있다.

김 위에 고등어회 한점, 양념밥 조금, 양파 무침 조금, 갈치젓갈 조금 올려서 먹으면 환상이다.

그리고 의외로 히트였던 고등어탕! 회만큼, 아니 회 보다 더 맛있었다. 뽀얀 국물이 인상적이었는데, 청양고추를 넣어서 칼칼한 국물이 진짜 제대로였다. 아니 이렇게 깊은 맛의 탕을 먹어본지가 언제였지! 시원한 배춧잎과 함께 보들한 고등어 살 한점, 국물까지 한입 떠먹는다. 너무 맛있다. 소주 한잔씩 하면서 국물만 홀짝홀짝 떠 먹어도 좋을 것 같았다.
고등어회가 정말 맛있습니다

김에 밥 고등어 양념 야채 싸먹으면
정말 최고입니다

고등어회 드실려면 여기로 가세요~~
이번 제주여행에서는 반드시 고등어회를 먹고자 하였는데, 망플에서 보고 그나마 조금이라도 평점이 더 높은 미영이네로 향했습니다.

바닷가에 있는 횟집이지만 예상 외로 굉장히 깔끔하고 메뉴판 정리도 잘 되어있는 것 같고, 밑반찬도

사진이 남아 있지 않아 아쉽지만 제주도 갈때 마다 맛있게 먹는딥. 양파무침과 고등어회를 김에 싸서 먹으면 환상. 양파무침이 양파냄새가 너무 나지도 않고 딱 좋다.
소자 (5만원, 2인분) 포장해 와서 먹었는데 정말 푸짐해서 도저히 혼자서 다 먹을 수는 없었다ㅠㅠ
고소한 고등어 회를 야채무침과 쌈장 찍은 마늘과 곁들여 제공되는 마른 김에 싸 먹으면 꿀맛!!! 회도 맛있지만 미나리랑 양파, 대파 등을 새콤매콤하게 무친 야채무침이 진짜 별미였다. 느끼함 박멸!
그리고 이 집의 킬포는 뭐니뭐니해도 고등어탕!!! 말간 국물이 매우 찐~한데, 들깨가루를 쓰지 않고 미영이네의 비법으로 조리했다고 한다. 약간의 텁텁하면서도 감칠맛으로 미루어 보아 생선을 갈아 넣은 게 아닌가 추측해봄... 여튼 뜨끈하게 데워 먹으니 진짜 술이 술술 들어가는 맛!
워낙 양도 푸짐하고 (2인분을 혼자 텨묵했으니^_ㅠ) 회랑 탕 자체가 맛있어서 밑반찬에는 거의 손이 가지 않았는데, 갈치젓은 제범 콤콤하고 비리면서 짭쪼롬한 게 맛있었음!
고등어회로 평점 높은 곳으로 골랐어요
역시나 5:30?? 일찍갔더니 웨이팅은 없지만 안에 사람들은 꽤 많았고 먹고나오니 웨이팅 있더라구여
2인분 시켰는데 괜추냇어요
음 근데 전에 다른집에서 먹었을때랑은 좀 음 그냥 존맛탱 정도는 아녔어여

화장실은 남녀공용 이라 좀 당황스럽
히든클리프호텔 풀파티를 포기하고 친구한테 끌려간 미영이네. 근데 한점 먹자마자 대만족!
풀파티 포기하길 잘했어 한 맛
양념된 채소와 싸먹는 것도 맛있지만 개인적으로 갈치속젓이랑 같이 먹으니까 느끼할수있는 고등어회를 끝까지 물리지 않고 맛있게 먹을수있었음 아직도 생각나는 맛..
그리고 매운탕은 빨개야지 하는 생각이라 지리는 기대를 아예 안하고있었는데 진짜 맛있습니다
들깨가루를 넣지않았다는데 너무 고소하고 살짝 얼큰하기도하고..
제주도 먹부림 다녀왔는데 그중에서도 제일 기억에 남는 식당!
고등어회 소-5만원
양념이랑 먹어도 맛있지만 젓갈이랑 더 맛있고
같이 주시는 밥이랑 김이랑 싸먹어도 맛있지만
백

,title,Point,View,Review,Star,Type,Menu,review
0,미영이네식당,4.7,"237,088",103,"5,042",해산물 요리,"[고등어회 (대) + 탕, 고등어회 (소) + 탕, 고등어구이]",[월드베스트 고등어회다. 먹선생 기준 미슐랭 3스타. 제주도 가면 반드시 가야하는 ...


{0: {'title': '미영이네식당',
  'Point': '4.7',
  'View': '237,088',
  'Review': '103',
  'Star': '5,042',
  'Type': '해산물 요리',
  'Menu': ['고등어회 (대) + 탕', '고등어회 (소) + 탕', '고등어구이'],
  'review': ['월드베스트 고등어회다. 먹선생 기준 미슐랭 3스타. 제주도 가면 반드시 가야하는 식당, 아니? 여기 고등어회 먹으러 제주도 또갈거다.\n-\n제주 서귀포시 모슬포 #미영이네 \n-\n미영이누나(?)에게 사랑을 고백하고싶어지는 곳이에요. 스토리에 올리자마자 디엠이 폭발했었죠. 부럽다부터 시작해서 여기가 내 인생고등어회라는 말까지 다들 미영이네에 대한 사랑이 묻어나는 메세지를 보내주시더라구요.\n-\n#고등어회\n한마디로 그냥 “인생고등어회”입니다. 맛있어요. 뭔지 모르겠는데 그냥 맛있어요. 아무것도 안찍어먹어도 맛있고 각종 소스에 곁들여먹어도 맛있어요. 맛있다는 말밖에할 수 없어 죄송하지만 그냥 맛있는걸 어떡해요?\n-\n인기가 많은 이유가 있어요. 단지 맛있다는 이유만으로도 충분한데 모든 직원분들이 각자의 개성대로 친절하고 유쾌하셔서 식사하는 시간 내내 기분이 좋았습니다.\n-\n당시 저녁시간보다 살짝 일찍 맞춰 갔음에도 불구하고 마감이였는데 감사하게도 먹선생은 막차타고 문닫고들어가는 행운아.\n-\n고등어회가 정말 맛있고 술이랑 같이 먹으면 한라산 정상이 제 뱃속이 들어있다고 할 정도로 한도끝도없이 들어갈것 같습니다. 운전때문에 참았어요. 여러분 미영이네 갈때는 꼭 택시타고 가세요.\n-\n아참. 고등어탕이 진짜 맛있어요. 맑은 탕으로 나오는데 하나도 비리지 않고 정말 깊은 맛이 납니다. 다음 제주도 방문때는 미영이네에 오기위해 숙소를 미영이네 근처로 잡으려구요. 최소한 서귀포에라도.\n-\n총평\nWORLD BEST',
   '고등어회\n원래 가던 곳보다 두껍게 썰어준다\n고등어탕은 처음 먹어보는데 고소하고 맛있다\n앞에 바로

In [20]:
# 음식점 클릭
i=0
j=0
element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
driver.find_element(By.CSS_SELECTOR,element).click()
time.sleep(0.5)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child(0) > div:nth-child(0) > figure > a > div > img"}
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x011478B3+2193587]
	Ordinal0 [0x010E0681+1771137]
	Ordinal0 [0x00FF41A8+803240]
	Ordinal0 [0x010224A0+992416]
	Ordinal0 [0x0102273B+993083]
	Ordinal0 [0x0104F7C2+1177538]
	Ordinal0 [0x0103D7F4+1103860]
	Ordinal0 [0x0104DAE2+1170146]
	Ordinal0 [0x0103D5C6+1103302]
	Ordinal0 [0x010177E0+948192]
	Ordinal0 [0x010186E6+952038]
	GetHandleVerifier [0x013F0CB2+2738370]
	GetHandleVerifier [0x013E21B8+2678216]
	GetHandleVerifier [0x011D17AA+512954]
	GetHandleVerifier [0x011D0856+509030]
	Ordinal0 [0x010E743B+1799227]
	Ordinal0 [0x010EBB68+1817448]
	Ordinal0 [0x010EBC55+1817685]
	Ordinal0 [0x010F5230+1856048]
	BaseThreadInitThunk [0x776CFA2

In [13]:
ex = driver.find_elements(By.CSS_SELECTOR,'p.RestaurantReviewItem__ReviewText')

In [15]:
for i in ex:
    print(i.text)

월드베스트 고등어회다. 먹선생 기준 미슐랭 3스타. 제주도 가면 반드시 가야하는 식당, 아니? 여기 고등어회 먹으러 제주도 또갈거다.
-
제주 서귀포시 모슬포 #미영이네 
-
미영이누나(?)에게 사랑을 고백하고싶어지는 곳이에요. 스토리에 올리자마자 디엠이 폭발했었죠. 부럽다부터 시작해서 여기가 내 인생고등어회라는 말까지 다들 미영이네에 대한 사랑이 묻어나는 메세지를 보내주시더라구요.
-
#고등어회
한마디로 그냥 “인생고등어회”입니다. 맛있어요. 뭔지 모르겠는데 그냥 맛있어요. 아무것도 안찍어먹어도 맛있고 각종 소스에 곁들여먹어도 맛있어요. 맛있다는 말밖에할 수 없어 죄송하지만 그냥 맛있는걸 어떡해요?
-
인기가 많은 이유가 있어요. 단지 맛있다는 이유만으로도 충분한데 모든 직원분들이 각자의 개성대로 친절하고 유쾌하셔서 식사하는 시간 내내 기분이 좋았습니다.
-
당시 저녁시간보다 살짝 일찍 맞춰 갔음에도 불구하고 마감이였는데 감사하게도 먹선생은 막차타고 문닫고들어가는 행운아.
-
고등어회가 정말 맛있고 술이랑 같이 먹으면 한라산 정상이 제 뱃속이 들어있다고 할 정도로 한도끝도없이 들어갈것 같습니다. 운전때문에 참았어요. 여러분 미영이네 갈때는 꼭 택시타고 가세요.
-
아참. 고등어탕이 진짜 맛있어요. 맑은 탕으로 나오는데 하나도 비리지 않고 정말 깊은 맛이 납니다. 다음 제주도 방문때는 미영이네에 오기위해 숙소를 미영이네 근처로 잡으려구요. 최소한 서귀포에라도.
-
총평
WORLD BEST
고등어회
원래 가던 곳보다 두껍게 썰어준다
고등어탕은 처음 먹어보는데 고소하고 맛있다
앞에 바로 항구가 있어서 진짜 섬에 온 느낌이 나서 좋았다
제주도 갈때마다 방문해서... 몇번째 방문인지도 모르겠는 미영이네. 근데 정말 갈때마다 맛있다ㅠ.ㅠ

친구랑 내가 고등어 회를 너무 좋아해서 제주 여행다니다가도 좀 괜찮다는 곳 있으면 들어가서 먹어보고 했는데 도대체 미영이네보다 맛있는 곳이 없었음.. 이유가 뭔데 여기 고등어는 이렇게 기름지고 고소하고 맛잇는지가 너무 궁금하다...!